In [16]:
import requests, datetime, pickle

URL = 'https://open-api.bingx.com'

descargar_data = True

# Obtener la serie de precios para BTC, ETH y XRP 

En temporalidades: 1d y/o 4h, 2h, 1h, 30m, 15m

data_raw = {<br />
    'BTC': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'ETH': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'XRP':<br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    }<br />
}<br />

In [17]:
from backtesting.fechas import past_timestamp, now_timestamp, retrasar_tres_horas
from backtesting.data import response_to_dataframe
from backtesting.ficheros import crear_directorio

In [18]:
if not descargar_data:
    with open('./data/data_raw.pickle', 'rb') as f:
        data_raw = pickle.load(f)
    activos = list(data_raw.keys())
    temporalidades = list(data_raw[activos[0]].keys())

else:
    path = '/openApi/swap/v2/quote/klines'

    data_1d = True
    data_low_temporality = True

    temporalidades = []
    if data_1d:
        temporalidades.append('1d')
    if data_low_temporality:
        temporalidades.append('4h')
        temporalidades.append('2h')
        temporalidades.append('1h')
        temporalidades.append('30m')
        temporalidades.append('15m')
    if not data_1d and not data_low_temporality:
        exit()

    now = datetime.datetime.now()
    start_time = []
    for t in temporalidades:
        if t == '1d':
            start_time.append(int(past_timestamp(365, 'days', now)))
        elif t == '4h':
            start_time.append(int(past_timestamp(45, 'days', now)))
        elif t == '2h':
            start_time.append(int(past_timestamp(20, 'days', now)))
        elif t == '1h':
            start_time.append(int(past_timestamp(20, 'days', now)))
        elif t == '30m':
            start_time.append(int(past_timestamp(6, 'days', now)))
        elif t == '15m':
            start_time.append(int(past_timestamp(3, 'days', now)))

    limit = 555
    activos = ['BTC', 'ETH', 'XRP', 'ENS']
    service = URL + path
    endTime = int(datetime.datetime.timestamp(datetime.datetime.now()) * 1000)
    data_raw = {}
    for activo in activos:
        data_raw[activo] = {}
        symbol = activo + '-USDT'
        for i, interval in enumerate(temporalidades):
            response = requests.get(service, params={'symbol':symbol, 'interval':interval, 'limit':limit, 'startTime':start_time[i], 'endTime':endTime})
            df = response_to_dataframe(response.json())
            if df is None:
                continue
            df['time'] = df['time'].apply(retrasar_tres_horas)
            df = df.set_index('time')
            data_raw[activo][interval] = df

    crear_directorio('data')
    with open('./data/data_raw.pickle', 'wb') as f:
        pickle.dump(data_raw, f, pickle.HIGHEST_PROTOCOL)

# Aplicar Squeeze Momentum Indicator a cada serie temporal y obtener la temporalidad que tengan alta correlacion para cada activo

SMI_default = {<br />
    'BTC': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'ETH': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'XRP':<br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    }<br />
<br />
}<br />
<br />
SMI_default_corr = {<br />
    'BTC': <br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    },<br />
    'ETH': <br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    },<br />
    'XRP':<br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    }<br />
<br />
}<br />

In [19]:
from backtesting.indicadores import squeeze_momentum_indicator
from scipy.stats import pearsonr

In [20]:
SMI_default = {}
for activo in activos:
    SMI_default[activo] = {}
    for t in temporalidades:
        df = data_raw[activo][t].iloc[:]
        df = squeeze_momentum_indicator(df)
        SMI_default[activo][t] = df.dropna()
        

In [21]:
SMI_default_corr = {}
for activo in activos:
    SMI_default_corr[activo] = {}
    for t in temporalidades:
        df = SMI_default[activo][t].iloc[:]
        df = df[['value', 'close']]
        SMI_default_corr[activo][t], _ = pearsonr(df['value'], df['close'])

print('Correlaciones')
for activo in activos:
    for t in temporalidades:
        print(f'{activo} {t} {round(SMI_default_corr[activo][t], 2)}')
    print()

Correlaciones
BTC 1d 0.36
BTC 4h 0.56
BTC 2h 0.53
BTC 1h 0.41
BTC 30m 0.32
BTC 15m 0.51

ETH 1d 0.41
ETH 4h 0.47
ETH 2h 0.56
ETH 1h 0.52
ETH 30m 0.4
ETH 15m 0.34

XRP 1d 0.41
XRP 4h 0.33
XRP 2h 0.43
XRP 1h 0.28
XRP 30m -0.01
XRP 15m 0.42

ENS 1d 0.44
ENS 4h 0.4
ENS 2h 0.42
ENS 1h 0.34
ENS 30m 0.18
ENS 15m 0.41



In [22]:
temporalidades_filtradas = {}
for activo in activos:
    temporalidades_filtradas[activo] = {}
    for t in temporalidades:
        if SMI_default_corr[activo][t] > 0.5:
            temporalidades_filtradas[activo][t] = SMI_default_corr[activo][t]

print('Temporalidades filtradas')
for activo in temporalidades_filtradas:
    for t in temporalidades_filtradas[activo]:
        print(f'correlacion {activo} {t}: {round(temporalidades_filtradas[activo][t], 2)}')
    print()

Temporalidades filtradas
correlacion BTC 4h: 0.56
correlacion BTC 2h: 0.53
correlacion BTC 15m: 0.51

correlacion ETH 2h: 0.56
correlacion ETH 1h: 0.52





### conclusiones 1:

- Si optimizo los parámetros para 1d, como afectan las demás temporalidades?
- Si optimizo los parámetros para una temporalidad con alta correlación, como afectan las demás temporalidades?
- Que diferencia existe entre aplicar la estrategia con parámetros por default y optimizados?

# Aplicar las condiciones de la estrategia a las temporalidades más correlacionadas y visualizar las últimas entradas

In [35]:
from backtesting.indicadores import backtest_entry
import plotly.graph_objects as go

In [24]:
entradas_backtest = {}
for activos in temporalidades_filtradas:
    for t in temporalidades_filtradas[activos]:
        entrada_short, entrada_long = backtest_entry(SMI_default[activos][t])
        dict_key = activos + '_' + t
        if len(entrada_short) > 0:
            dict_key_2 = dict_key + '_short'
            entradas_backtest.update({dict_key_2: entrada_short})
        if len(entrada_long) > 0:
            dict_key_2 = dict_key + '_long'
            entradas_backtest.update({dict_key_2: entrada_long})
        

In [60]:
## Obtenemos los dataframes de los activos y temporalidades filtradas, con indicador aplicado y parámetros default
## Visualizar las entradas generadas

ultimas_velas = 100

for activo in SMI_default:
    if len(temporalidades_filtradas[activo]) > 0:
        for t in temporalidades_filtradas[activo]:
            # print(f'{activo} {t}')
            # display(SMI_default[activo][t])
            # print()
            df = SMI_default[activo][t].iloc[-ultimas_velas:]

            candlestick = go.Candlestick(x=df.index,
                            open=df['open'],
                            high=df['high'],
                            low=df['low'],
                            close=df['close'])

            upper_band = go.Scatter(x=df.index, y=df['upper_BB'], name='upper_band', line=dict(color='red', width=1))
            lower_band = go.Scatter(x=df.index, y=df['lower_BB'], name='lower_band', line=dict(color='red', width=1))
            
            upper_KC = go.Scatter(x=df.index, y=df['upper_KC'], name='upper_KC', line=dict(color='blue', width=1, dash='dash'))
            lower_KC = go.Scatter(x=df.index, y=df['lower_KC'], name='lower_KC', line=dict(color='blue', width=1, dash='dash'))
            
            bar = go.Bar(x=df.index, y=df['value'], name='squeeze')


            fig = go.Figure(data=[candlestick, upper_band, lower_band, upper_KC, lower_KC])
            fig2 = go.Figure(data=[bar])
            
            fig.layout.xaxis.type = 'category'
            fig.layout.xaxis.rangeslider.visible = False
            # update figure 1
            fig.update_layout(
                autosize=False,
                width=1500,
                height=600,
                margin=dict(
                    
                    l=50,
                    r=50,
                    b=100,
                    t=50,
                    pad=4
                ),
                paper_bgcolor="LightSteelBlue",
                title=f'{activo} {t}'
            )
            # update figure 2
            fig2.update_layout(
                autosize=False,
                width=1500,
                height=300,
                margin=dict(
                    
                    l=50,
                    r=150,
                    b=1,
                    t=10,
                    pad=4
                ),
                paper_bgcolor="LightSteelBlue",
            )

            for k in entradas_backtest:
                if k.startswith(f'{activo}_{t}') and k.endswith('short'):
                    for x_value in entradas_backtest[f'{activo}_{t}_short'].index:
                        if x_value in df.index:
                            fig.add_shape(
                                type="line",
                                x0=x_value,
                                y0=df['low'].min(),
                                x1=x_value,
                                y1=df['high'].max(),
                                line=dict(color="red", width=1, dash="dash"),
                            )
                            fig2.add_shape(
                                type="line",
                                x0=x_value,
                                y0=df['value'].min(),
                                x1=x_value,
                                y1=df['value'].max(),
                                line=dict(color="red", width=1, dash="dash"),
                            )

                if k.startswith(f'{activo}_{t}') and k.endswith('long'):
                    for x_value in entradas_backtest[f'{activo}_{t}_long'].index:
                        if x_value in df.index:
                            fig.add_shape(
                                type="line",
                                x0=x_value,
                                y0=df['low'].min(),
                                x1=x_value,
                                y1=df['high'].max(),
                                line=dict(color="green", width=1, dash="dash"),
                            )
                            fig2.add_shape(
                                type="line",
                                x0=x_value,
                                y0=df['value'].min(),
                                x1=x_value,
                                y1=df['value'].max(),
                                line=dict(color="green", width=1, dash="dash"),
                            )

            fig.show()
            fig2.show()            

# Optimizar los parámetros del SMI para la temporalidad DIARIA y verificar la correlacion en las otras temporalidades

SMI_optimized = {<br />
    'BTC': <br />
    {'best_temporality': pd.DataFrame()},<br />
    'ETH': <br />
    {'best_temporality': pd.DataFrame()},<br />
    'XRP':<br />
    {'best_temporality': pd.DataFrame()}<br />
<br />
}<br />

In [46]:
entradas_backtest['BTC_4h_short']

,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
time,,,,,,,,,,,,
2023-04-14 05:00:00,30739.0,30803.0,30829.0,30661.0,6051.73,30756.549496,29639.300504,168.0,30679.127463,29716.722537,429.032143,False
2023-04-14 17:00:00,30321.5,30464.5,30510.0,30270.5,4071.54,30796.903253,29665.696747,239.5,30782.901499,29679.698501,213.856786,False


In [57]:
activo = 'BTC'
t = '4h'
df = SMI_default[activo][t].iloc[:]

candlestick = go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])

upper_band = go.Scatter(x=df.index, y=df['upper_BB'], name='upper_band', line=dict(color='red', width=1))
lower_band = go.Scatter(x=df.index, y=df['lower_BB'], name='lower_band', line=dict(color='red', width=1))

upper_KC = go.Scatter(x=df.index, y=df['upper_KC'], name='upper_KC', line=dict(color='blue', width=1, dash='dash'))
lower_KC = go.Scatter(x=df.index, y=df['lower_KC'], name='lower_KC', line=dict(color='blue', width=1, dash='dash'))

bar = go.Bar(x=df.index, y=df['value'], name='squeeze')


fig = go.Figure(data=[candlestick, upper_band, lower_band, upper_KC, lower_KC])
fig.layout.xaxis.rangeslider.visible = False
## agregamos una linea vertical

for k in entradas_backtest:
    if k.startswith(f'{activo}_{t}') and k.endswith('short'):
        for x_value in entradas_backtest[f'{activo}_{t}_short'].index:
            if x_value in df.index:
                fig.add_shape(
                    type="line",
                    x0=x_value,
                    y0=df['low'].min(),
                    x1=x_value,
                    y1=df['high'].max(),
                    line=dict(color="red", width=1, dash="dash"),
                )

    if k.startswith(f'{activo}_{t}') and k.endswith('long'):
        for x_value in entradas_backtest[f'{activo}_{t}_long'].index:
            if x_value in df.index:
                fig.add_shape(
                    type="line",
                    x0=x_value,
                    y0=df['low'].min(),
                    x1=x_value,
                    y1=df['high'].max(),
                    line=dict(color="green", width=1, dash="dash"),
                )

fig.show()


In [25]:
## TODO: Esto de acá fue una prueba y no es relevante para la función del módulo actual
for señales in entradas_backtest:
    print(señales)
    display(entradas_backtest[señales].iloc[-1].to_frame().T)
    print()

BTC_4h_short


,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
2023-04-14 17:00:00,30321.5,30464.5,30510.0,30270.5,4071.54,30796.903253,29665.696747,239.5,30782.901499,29679.698501,213.856786,False



BTC_2h_short


,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
2023-05-03 23:00:00,29072.5,29099.0,29192.0,28993.0,2587.03,29139.274285,28083.175715,199.0,29113.755394,28108.694606,357.014821,False



BTC_15m_long


...

,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
2023-05-08 01:00:00,1865.16,1867.74,1869.47,1857.3,27039.66,1929.157229,1857.967771,12.17,1927.496845,1859.628155,-12.386554,False



ETH_1h_short


,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
2023-05-10 21:00:00,1840.76,1831.16,1841.71,1826.87,17985.6,1879.354212,1818.598788,14.84,1877.143245,1820.809755,1.2995,False



ETH_1h_long


,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
2023-05-08 19:00:00,1839.75,1846.82,1846.9,1838.67,8563.36,1877.407811,1826.431189,8.23,1875.383811,1828.455189,-10.882082,False


# Aplicar los criterios de la estrategia para cada configuración del indicador y visualizar las señales